# Predicción juegos de rugby

## Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
%matplotlib inline
pd.set_option('display.max_columns', None)


In [2]:
# ! pip install bs4
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from html import unescape

## Primer acercamiento con la librería BeautifulSoup a un partido en específico

Documentación librería
https://beautiful-soup-4.readthedocs.io/en/latest/

Código tomado y modificados de los pasos de la página: https://www.datacamp.com/es/tutorial/web-scraping-using-python

In [3]:
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
html = urlopen(url)


In [4]:
soup = BeautifulSoup(html, 'lxml')
type(soup)


bs4.BeautifulSoup

In [5]:
# Tener el título de la página
title = soup.title
print(title)


<title>Statistiques de Stade Toulousain - Section Paloise - J15 - 2025-2026 | Top 14 - Site Officiel</title>


In [6]:
rows = soup.find_all('div')
# print(rows)


In [7]:
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'es-ES,es;q=0.9',
    'Referer': 'https://top14.lnr.fr/'
}

response = requests.get(url, headers=headers)

print(f"Estado de la respuesta: {response.status_code}")
print(f"Longitud del contenido: {len(response.text)} caracteres")
print(f"Primeros 500 caracteres del HTML:\n{response.text[:500]}")


soup = BeautifulSoup(response.text, 'html.parser')
scripts = soup.find_all('script')
print(f"\nTotal de etiquetas <script> encontradas: {len(scripts)}")

Estado de la respuesta: 200
Longitud del contenido: 161705 caracteres
Primeros 500 caracteres del HTML:
<!DOCTYPE html>
<html lang="fr">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />
  <meta name="csrf-token" content="1X7MUCeW1YGYVTpUV0wJ58Fzqja8SGMxFtQVwgKE" />

  <title>Statistiques de Stade Toulousain - Section Paloise - J15 - 2025-2026 | Top 14 - Site Officiel</title>
  <meta name="title" content="Statistiques de Stade Toulousain - Section Paloise - J15 - 2025-2026 | Top 14 - Site Officiel" />
  <met

Total de etiquetas <script> encontradas: 8


In [8]:
# <div class="match-header__season-day">
#                   Match terminé - J15 - 24/01/2026 - 21h00
#               </div>

#Si quiero extraer la fecha

url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
fecha = soup.find('div', class_='match-header__season-day')
fecha = fecha.get_text(strip=True) if fecha else "N/A"
fecha = "-".join(fecha.split("-")[2:])
fecha


' 24/01/2026 - 21h00'

## Extracción estadísticas del partido

In [15]:
# URL del partido
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

#Función para extraer estadísticas del partido
def extraer_resumen_partido(url):
    print(f" Extrayendo datos generales de: {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Obtener el Resultado Final
    marcador_tag = soup.find('div', class_='title--large')
    marcador = marcador_tag.get_text(strip=True) if marcador_tag else "N/A"

    # Obtener fecha
    fecha = soup.find('div', class_='match-header__season-day')
    fecha = fecha.get_text(strip=True) if fecha else "N/A"
    partes = fecha.split("-")
    fecha = "-".join(partes[-2:]).strip()
    formato = "%d/%m/%Y - %Hh%M"
    fecha = datetime.strptime(fecha, formato)

    # Identificar Equipos
    equipo_local = soup.find_all('a', class_='match-header-club__title')[0].get_text(strip=True)
    equipo_visitante = soup.find_all('a', class_='match-header-club__title')[1].get_text(strip=True)

    print(f"Resultado Final: {equipo_local} {marcador} {equipo_visitante}, Fecha: {fecha}")

    # Extraer grupos de estadísticas (Points, Domination, Faltas, etc.)
    grupos = soup.find_all('div', class_='match-statistics__group')

    stats_totales = []

    for grupo in grupos:
        # Extraer el título de la categoría
        categoria = grupo.find('h2', class_='match-statistics__title')
        if not categoria: continue
        nombre_categoria = categoria.get_text(strip=True)

        # Extraer cada barra de estadística dentro del grupo
        filas = grupo.find_all('div', class_='stats-bar')
        for fila in filas:
            metrica = fila.find('div', class_='stats-bar__title').get_text(strip=True)
            valor_local = fila.find('div', class_='stats-bar__val--left').get_text(strip=True)
            valor_visitante = fila.find('div', class_='stats-bar__val--right').get_text(strip=True)

            stats_totales.append({
                'Categoría': nombre_categoria,
                'Métrica': metrica,
                equipo_local: valor_local,
                equipo_visitante: valor_visitante
            })

        df_match_stats_t=pd.DataFrame(stats_totales)
        df_match_stats_t['categoria_metrica']=df_match_stats_t['Categoría']+'_'+df_match_stats_t['Métrica']
        df_match_stats_t.drop(columns=['Categoría', 'Métrica'], inplace=True)
        marcador_l = "-".join(marcador.split("-")[:1])
        marcador_v = "-".join(marcador.split("-")[1:])
        marcadores=[marcador_l, marcador_v, 'Resultado']
        oponente=[equipo_visitante, equipo_local , 'Oponente']
        df_match_stats_t.loc[len(df_match_stats_t)] = marcadores
        df_match_stats_t.loc[len(df_match_stats_t)] = oponente
        df_match_stats_t=df_match_stats_t.set_index('categoria_metrica').T
        df_match_stats_t['fecha']=fecha
        df_match_stats_t=df_match_stats_t.reset_index()
        df_match_stats_t = df_match_stats_t.rename(columns={'index': 'equipo'})
    return df_match_stats_t


df_match_stats = extraer_resumen_partido(url)

if not df_match_stats.empty:
    display(df_match_stats)
    df_match_stats.to_csv('resumen_global_partido.csv', index=False)
else:
    print("No se pudieron extraer las estadísticas grupales.")

 Extrayendo datos generales de: https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match
Resultado Final: Stade Toulousain 59 - 22 Section Paloise, Fecha: 2026-01-24 21:00:00


categoria_metrica,equipo,Points_Essais accordés,Domination_Possession de la balle,Domination_Occupation,Domination_Possession dans son camp,Domination_Possession dans le camp adverse,Domination_Possession 22m adverses,Actions_Mêlées obtenues,Actions_Mêlées perdues,Actions_Mêlées gagnées,Actions_Mêlées refaites,Actions_Touches obtenues,Actions_Touches gagnées sur son propre lancer,Actions_Touches gagnées sur lancer adverse,Fautes_En-avant commis,Fautes_Pénalités réussies,Fautes_Pénalités concédées,Plaquages et ballons joués_Plaquages réussis,Plaquages et ballons joués_Plaquages offensifs réussis,Plaquages et ballons joués_Plaquages manqués,Plaquages et ballons joués_Ballons joués au pied,Plaquages et ballons joués_Ballons passés,Resultado,Oponente,fecha
0,Stade Toulousain,9,57 %,53 %,43 %,48 %,50 %,8,0,7,1,14,10,1,4,0,8,120,5,16,19,168,59,Section Paloise,2026-01-24 21:00:00
1,Section Paloise,3,43 %,47 %,57 %,52 %,50 %,6,1,4,1,14,13,1,7,1,11,152,3,26,20,95,22,Stade Toulousain,2026-01-24 21:00:00


## Extracción estadísticas de los jugadores

Código corregido con asistente Gemini

In [17]:
# URL del partido (Toulouse vs Pau)
url = "https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

#Función para extraer estadísticas de los jugadores del partido
def extraer_rugby_data_jugadores(url):
    print(f"Iniciando extracción de jugadores en: {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar los componentes <players-ranking>
    ranking_tags = soup.find_all('players-ranking')

    if len(ranking_tags) < 2:
        print("No se encontraron los datos de los jugadores. Verifica la conexión.")
        return None

    # Obtener fecha
    fecha = soup.find('div', class_='match-header__season-day')
    fecha = fecha.get_text(strip=True) if fecha else "N/A"
    partes = fecha.split("-")
    fecha = "-".join(partes[-2:]).strip()
    formato = "%d/%m/%Y - %Hh%M"
    fecha = datetime.strptime(fecha, formato)

    all_players = []

    # Nombres de los equipos
    equipo_local = soup.find_all('a', class_='match-header-club__title')[0].get_text(strip=True)
    equipo_visitante = soup.find_all('a', class_='match-header-club__title')[1].get_text(strip=True)
    equipos = [equipo_local, equipo_visitante]

    for idx, tag in enumerate(ranking_tags):
        raw_json = tag.get(':ranking')
        if raw_json:
            decoded_json = unescape(raw_json)
            data_list = json.loads(decoded_json)

            for p in data_list:
                all_players.append({
                    'Equipo': equipos[idx],
                    'Nombre': p['player']['name'],
                    'Posicion': p['position'],
                    'Minutos': int(p['minutesPlayed']),
                    'Puntos': int(p['nbPoints']),
                    'Essais': int(p['nbEssais']),
                    'Offload': int(p['offload']),
                    'Franchissement': int(p['lineBreak']),
                    'Tackles_Ok': int(p['totalSuccessfulTackles']),
                    'Tarjetas_Amarillas': int(p['nbCartonsJaunes'])
                })

    all_players=pd.DataFrame(all_players)
    all_players['fecha']=fecha

    return all_players

# EJECUCIÓN
df_final = extraer_rugby_data_jugadores(url)

if df_final is not None:
    print(f"Se cargaron {len(df_final)} jugadores.")
    display(df_final.sort_values(by='Puntos', ascending=False).head(10))


Iniciando extracción de jugadores en: https://top14.lnr.fr/feuille-de-match/2025-2026/j15/11409-toulouse-pau/statistiques-du-match
Se cargaron 46 jugadores.


,Equipo,Nombre,Posicion,Minutos,Puntos,Essais,Offload,Franchissement,Tackles_Ok,Tarjetas_Amarillas,fecha
18,Stade Toulousain,Thomas RAMOS,Demi d'ouverture,80,14,0,2,0,6,0,2026-01-24 21:00:00
26,Section Paloise,F. BRAU-BOIRIE,Centre,80,10,2,0,1,10,0,2026-01-24 21:00:00
31,Section Paloise,A. DESPERES RIGOU,Demi d'ouverture,68,7,0,0,0,8,0,2026-01-24 21:00:00
4,Stade Toulousain,G. COLOMBE REAZEL,Centre,26,5,1,0,1,2,0,2026-01-24 21:00:00
0,Stade Toulousain,Dorian ALDEGHERI,Pilier,54,5,1,0,1,7,0,2026-01-24 21:00:00
2,Stade Toulousain,Joshua BRENNAN,Troisième ligne aile,47,5,1,1,0,8,0,2026-01-24 21:00:00
14,Stade Toulousain,Julien MARCHAND,Talonneur,54,5,1,2,0,9,0,2026-01-24 21:00:00
37,Section Paloise,Thomas LACLAYAT,Pilier,51,5,1,0,0,7,0,2026-01-24 21:00:00
16,Stade Toulousain,E. MEAFOU,2ème ligne,63,5,1,1,0,8,0,2026-01-24 21:00:00
22,Stade Toulousain,Jack WILLIS,Centre,33,5,1,0,2,8,0,2026-01-24 21:00:00


## Extracción juegos temporadas


In [11]:
journee=['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14',
         'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26',
         'barrage', 'access-top-14', 'demi-finale', 'finale']
saisons=['2025-2026', '2024-2025', '2023-2024', '2022-2023', '2021-2022']

urls_journees=[]

# Creacion de los url por jornada y temporada a buscar para extraer las url de los partidos
for s in saisons:
  for j in journee:
    url = f"https://top14.lnr.fr/calendrier-et-resultats/{s}/{j}"
    urls_journees.append(url)

urls_match=[]

# Busqueda de las url de cada partido en cada jornada
for url in urls_journees:
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    link_match = [link.get('href') for link in soup.find_all('a', class_='match-line__score')]
    urls_match.append(link_match)
    time.sleep(5)

# Volver el resultado en un vector para exportarlo y tener la base de las url por partido
urls_planas = [match_url for jornada in urls_match for match_url in jornada]
urls_planas = [url for url in urls_planas if url]

separator="/"
urls_scarp=[]
for url in urls_planas:
    ult_s=separator.join([url, 'statistiques-du-match'])
    urls_scarp.append(ult_s)

urls_scarp_pd=pd.DataFrame(urls_scarp, columns=['url_match'])
print("Total de partidos encontrados: ", len(urls_scarp_pd))
urls_scarp_pd.to_csv('urls_scrapping_top14.csv', index=False)

Total de partidos encontrados:  870


# Extracción de datos de los juegos recuperados.  

In [ ]:
datos_partidos_top14=[]
datos_jugadores_top14=[]

for url in urls_scarp:
    df_match_stats = extraer_resumen_partido(url)
    datos_partidos_top14.append(df_match_stats)
    df_jugadores=extraer_rugby_data_jugadores(url)
    datos_jugadores_top14.append(df_jugadores)
    time.sleep(2)

# Unificar todo
top14_partidos = pd.concat(datos_partidos_top14, ignore_index=True)
top14_jugadores = pd.concat(datos_jugadores_top14, ignore_index=True)

# Exportar a CSV
top14_partidos.to_csv('top14_partidos_2021_2026.csv', index=False)
top14_jugadores.to_csv('top14_jugadores_2021_2026.csv', index=False)

 Extrayendo datos generales de: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11311-paris-montauban/statistiques-du-match
Resultado Final: Stade Français Paris 47 - 24 US Montauban, Fecha: 2025-09-06 13:00:00
Iniciando extracción de jugadores en: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11311-paris-montauban/statistiques-du-match
 Extrayendo datos generales de: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11313-perpignan-bayonne/statistiques-du-match
Resultado Final: USA Perpignan 19 - 26 Aviron Bayonnais, Fecha: 2025-09-06 15:00:00
Iniciando extracción de jugadores en: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11313-perpignan-bayonne/statistiques-du-match
 Extrayendo datos generales de: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11308-castres-pau/statistiques-du-match
Resultado Final: Castres Olympique 15 - 17 Section Paloise, Fecha: 2025-09-06 17:00:00
Iniciando extracción de jugadores en: https://top14.lnr.fr/feuille-de-match/2025-2026/j1/11308-